In [1]:
import preprocess as pp
import configs
import persister
import relation
import lda
import vis
from utils import *

from collections import Counter
from pprint import pprint
from functools import reduce

import re
import numpy as np
from nltk.tree import Tree
from nltk import word_tokenize,sent_tokenize

not use stanford CoreNLP client!


[nltk_data] Error loading sentiwordnet: <urlopen error [Errno 111]
[nltk_data]     Connection refused>


In [2]:
# is_news = 1

# l = 7
# t = 4
# m = "c_v"
# i = 200
# min_df = 1

is_news = 0
l = 6
t = 14
m = "c_v"
i = 200
min_df = 1

# load
if is_news:
    _raw = persister.load_json(configs.RAWNEWS)
    parse = persister.read_parse()
    _input = persister.read_input(configs.NEWSINPUT)
    model_name = configs.NEWSMODEL.format(l, t, m, i, min_df)
    terms, doc_topic, topic_word = persister.read_lda(configs.NEWSLDA.format(model_name))
    model = persister.load_model(model_name)
    vec = persister.load_model(configs.NEWSVEC.format(min_df))
    print("load news")
else:
    _raw = persister.load_json(configs.RAWABSTRACT)
    _input = persister.read_input(configs.ABSTRACTINPUT)
    model_name = configs.ABSTRACTMODEL.format(l, t, m, i, min_df)
    terms, doc_topic, topic_word = persister.read_lda(configs.ABSTRACTLDA.format(model_name))
    parse = persister.read_parse(configs.ABSTRACTPARSE)
    model = persister.load_model(model_name)
    vec = persister.load_model(configs.ABSVEC.format(min_df))
    print("load abs")
tf = vec.fit_transform(_input) 
top_terms, top_docs = lda.get_topics(topic_word, terms, doc_topic, 20)
df_top_words, df_top_docs = lda.pd_topics_vis(top_terms, top_docs)

load abs


In [3]:
len(terms)

10453

In [4]:
distr = lda.get_dominant_topic(doc_topic)
print("docs distribution:",dict(Counter(distr)))

docs distribution: {9: 152, 5: 33, 11: 969, 12: 1330, 1: 7, 13: 3, 10: 76, 3: 3, 0: 8, 7: 14, 6: 1}


In [5]:
print("top terms info:")
df_top_words

top terms info:


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,"(gene, 123.1)","(drug, 96.87)","(expression, 78.19)","(disease, 77.49)","(protein, 75.45)","(biomedical, 64.39)","(pathway, 51.34)","(cell, 51.19)","(identify, 48.51)","(biological, 47.03)","(btm, 40.99)","(ad, 40.76)","(study, 39.19)","(apply, 37.74)","(manufacturing, 31.56)","(mrna, 29.42)","(relationship, 29.31)","(conclusion, 28.59)","(tool, 26.52)","(related, 25.93)"
Topic 1,"(patient, 223.06)","(medical, 143.69)","(health, 143.5)","(clinical, 139.72)","(treatment, 99.44)","(diagnosis, 86.78)","(disease, 81.07)","(risk, 79.0)","(record, 78.67)","(care, 68.06)","(friend, 47.64)","(healthcare, 42.31)","(cancer, 39.36)","(code, 36.28)","(data, 32.74)","(conclusion, 30.77)","(physician, 29.99)","(emr, 25.48)","(ehr, 25.13)","(doctor, 24.57)"
Topic 2,"(mirna, 42.64)","(permission, 25.17)","(quran, 13.82)","(holy, 9.76)","(aircraft, 8.0)","(provenance, 7.74)","(lv, 7.74)","(laplace, 7.05)","(ast, 6.96)","(chara, 6.65)","(yuru, 6.65)","(ldaclm, 6.22)","(privilege, 5.86)","(mmrm, 5.86)","(iexpand, 5.13)","(configure, 5.01)","(transcriptional, 5.0)","(pcfg, 4.14)","(tctm, 4.12)","(seqlda, 4.12)"
Topic 3,"(emotion, 181.73)","(drive, 87.04)","(style, 86.19)","(speech, 55.34)","(genre, 39.85)","(recognition, 38.09)","(speaker, 34.84)","(state, 33.33)","(write, 31.62)","(language, 30.16)","(acoustic, 30.1)","(driving, 29.08)","(driver, 27.25)","(essay, 26.57)","(sleep, 24.96)","(vehicle, 24.49)","(authorship, 22.67)","(noca, 22.09)","(author, 21.59)","(disorder, 21.36)"
Topic 4,"(sampling, 79.5)","(parallel, 73.15)","(gibbs, 67.68)","(security, 57.19)","(distribute, 52.59)","(collapse, 42.46)","(cluster, 33.26)","(bot, 32.4)","(brand, 31.0)","(large, 30.54)","(crime, 29.71)","(scale, 29.68)","(token, 29.34)","(approximation, 28.55)","(memory, 27.44)","(gpu, 27.32)","(sampler, 26.77)","(matrix, 23.32)","(atomic, 23.12)","(mh, 22.88)"
Topic 5,"(review, 537.88)","(sentiment, 474.31)","(opinion, 265.32)","(tweet, 235.55)","(product, 228.45)","(post, 206.48)","(online, 196.94)","(topic, 194.3)","(analysis, 180.66)","(twitter, 174.52)","(media, 171.56)","(aspect, 166.01)","(customer, 159.65)","(rating, 142.65)","(consumer, 111.75)","(social, 101.7)","(use, 101.33)","(negative, 88.63)","(positive, 85.19)","(study, 81.34)"
Topic 6,"(song, 81.12)","(audio, 74.02)","(music, 57.13)","(pn, 35.01)","(broadcast, 28.77)","(oov, 24.22)","(indonesian, 19.81)","(musical, 18.49)","(lyric, 17.49)","(estimate, 16.39)","(writer, 14.97)","(commonness, 14.53)","(submit, 13.59)","(hyperspectral, 11.69)","(eem, 10.71)","(mp, 10.39)","(isoform, 9.79)","(retrieve, 9.73)","(replicate, 9.62)","(timbre, 8.87)"
Topic 7,"(software, 268.18)","(question, 203.92)","(code, 149.7)","(developer, 143.61)","(source, 143.52)","(bug, 139.43)","(report, 120.26)","(project, 87.55)","(program, 68.98)","(answer, 65.81)","(repository, 53.22)","(requirement, 49.86)","(evolution, 49.8)","(technique, 48.97)","(open, 44.34)","(study, 43.24)","(lsi, 40.78)","(stack, 33.35)","(change, 30.56)","(nfr, 30.05)"
Topic 8,"(lm, 75.09)","(frame, 55.04)","(library, 31.56)","(asr, 28.23)","(street, 22.76)","(ldta, 18.42)","(adr, 18.41)","(rnn, 17.93)","(abbreviation, 15.75)","(adverse, 15.54)","(adapted, 15.29)","(lsm, 13.09)","(eligibility, 13.06)","(india, 12.3)","(attraction, 11.6)","(salience, 11.02)","(wall, 10.98)","(grams, 10.91)","(journal, 10.27)","(dntm, 10.12)"
Topic 9,"(topic, 893.39)","(research, 662.45)","(study, 393.56)","(analysis, 331.47)","(use, 254.09)","(article, 239.38)","(identify, 225.84)","(technology, 215.99)","(trend, 201.75)","(literature, 154.3)","(field, 150.61)","(text, 137.1)","(papers, 135.72)","(latent, 131.54)","(patent, 131.04)","(researcher, 130.56)","(scientific, 129.8)","(area, 125.64)","(dirichlet, 123.53)","(allocation, 122.98)"


In [6]:
print("top docs info:")
df_top_docs

top docs info:


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Topic 0,"(198, 0.548)","(263, 0.493)","(107, 0.407)","(1764, 0.386)","(2009, 0.383)","(1770, 0.371)","(1740, 0.361)","(894, 0.346)","(896, 0.334)","(65, 0.32)","(751, 0.318)","(725, 0.318)","(122, 0.314)","(52, 0.285)","(2577, 0.271)","(562, 0.264)","(526, 0.263)","(50, 0.257)","(685, 0.247)","(2248, 0.245)"
Topic 1,"(489, 0.751)","(142, 0.651)","(115, 0.587)","(722, 0.42)","(1566, 0.37)","(1101, 0.364)","(1112, 0.346)","(80, 0.346)","(32, 0.345)","(627, 0.339)","(415, 0.319)","(2304, 0.319)","(1037, 0.313)","(1446, 0.291)","(1402, 0.291)","(1129, 0.281)","(174, 0.277)","(64, 0.276)","(338, 0.275)","(1624, 0.266)"
Topic 2,"(1855, 0.23)","(1319, 0.208)","(1400, 0.199)","(872, 0.198)","(1309, 0.196)","(1079, 0.187)","(1340, 0.151)","(1310, 0.146)","(1164, 0.133)","(1350, 0.131)","(2158, 0.115)","(2386, 0.114)","(2188, 0.112)","(1242, 0.11)","(2220, 0.107)","(1868, 0.105)","(1932, 0.104)","(2422, 0.102)","(2457, 0.075)","(1876, 0.074)"
Topic 3,"(81, 0.685)","(874, 0.517)","(109, 0.374)","(771, 0.349)","(18, 0.338)","(195, 0.293)","(1178, 0.287)","(344, 0.285)","(1281, 0.265)","(1813, 0.255)","(171, 0.254)","(1422, 0.254)","(1761, 0.244)","(1306, 0.24)","(1266, 0.238)","(737, 0.233)","(1278, 0.228)","(113, 0.222)","(1175, 0.22)","(365, 0.216)"
Topic 4,"(508, 0.371)","(461, 0.346)","(413, 0.333)","(98, 0.282)","(1076, 0.27)","(720, 0.268)","(740, 0.268)","(741, 0.268)","(1666, 0.252)","(606, 0.248)","(693, 0.247)","(2027, 0.247)","(1379, 0.246)","(82, 0.241)","(2447, 0.231)","(886, 0.226)","(89, 0.225)","(1273, 0.217)","(450, 0.215)","(56, 0.206)"
Topic 5,"(85, 0.951)","(57, 0.727)","(386, 0.669)","(111, 0.651)","(1, 0.631)","(575, 0.57)","(434, 0.518)","(698, 0.513)","(1577, 0.493)","(786, 0.486)","(103, 0.48)","(59, 0.471)","(6, 0.466)","(270, 0.455)","(957, 0.454)","(2272, 0.452)","(67, 0.452)","(973, 0.451)","(410, 0.435)","(1936, 0.42)"
Topic 6,"(1570, 0.465)","(1117, 0.415)","(1606, 0.337)","(477, 0.302)","(757, 0.276)","(970, 0.254)","(1874, 0.252)","(1167, 0.169)","(1520, 0.166)","(684, 0.162)","(1668, 0.15)","(854, 0.147)","(2526, 0.144)","(702, 0.141)","(581, 0.141)","(1021, 0.131)","(2325, 0.126)","(2396, 0.125)","(1720, 0.124)","(995, 0.12)"
Topic 7,"(1586, 0.575)","(747, 0.526)","(224, 0.503)","(831, 0.447)","(453, 0.428)","(898, 0.427)","(20, 0.397)","(1594, 0.396)","(731, 0.392)","(2283, 0.386)","(380, 0.385)","(1473, 0.382)","(891, 0.379)","(1181, 0.377)","(2312, 0.376)","(2533, 0.375)","(945, 0.371)","(2467, 0.368)","(752, 0.357)","(141, 0.357)"
Topic 8,"(441, 0.215)","(1958, 0.173)","(890, 0.172)","(626, 0.172)","(2121, 0.167)","(2184, 0.165)","(70, 0.161)","(802, 0.16)","(1442, 0.157)","(273, 0.152)","(2561, 0.148)","(1343, 0.147)","(1440, 0.137)","(576, 0.136)","(2505, 0.129)","(2233, 0.125)","(1882, 0.123)","(849, 0.114)","(2522, 0.113)","(621, 0.108)"
Topic 9,"(2, 0.994)","(41, 0.9)","(1044, 0.871)","(317, 0.861)","(1013, 0.836)","(97, 0.813)","(996, 0.8)","(47, 0.798)","(1475, 0.772)","(335, 0.772)","(1305, 0.772)","(388, 0.765)","(308, 0.755)","(120, 0.752)","(546, 0.741)","(51, 0.737)","(8, 0.727)","(617, 0.724)","(264, 0.716)","(564, 0.712)"


In [7]:
# vis.pyLDA(topic_word, doc_topic, [len(s) for s in [word_tokenize(corp) for corp in _input]], vec.get_feature_names(), np.array(sum(tf).todense())[0])

In [8]:
t_idx = 1
for top_doc in top_docs[t_idx]:
    _idx = top_doc[0]
    if type(parse[_idx]) is str:
        print(_idx, "parse", "err")
        continue
    sents = relation.convert_parse2lemma_sents(parse[_idx])
    sort_idxs, importance, counts = relation.extract_important_sents(sents, [x[0] for x in top_terms[t_idx]], [x[1] for x in top_terms[t_idx]])
    for i in sort_idxs:
        if importance[i]>0:
            print(round(importance[i], 2), counts[i], _idx, i)
            sent_tokens = parse[_idx]["sentences"][i]["tokens"]
            sent_deps = parse[_idx]["sentences"][i]["enhancedPlusPlusDependencies"]
            print(" ".join([s["lemma"] for s in sent_tokens]))
            sents = sent_tokenize(_raw[_idx])
            if len(sents) > i:
                print(sents[i])
            else:
                print("sent token differ")
            triples = relation.extract_triples_from_sent(sent_deps, sent_tokens, True)
            for triple in triples:
                if None in triple:
                    print(triple)
                    continue
                s = [sent_tokens[i]["originalText"] for i in triple[0]]
                p = [sent_tokens[i]["originalText"] for i in triple[1]]
                o = [sent_tokens[i]["originalText"] for i in triple[2]]
                print("{}-{}-{}".format(s,p,o))
            print("="*40)

1.07 2 489 5
lda show that , prior to surgery , patient ' priority be primarily in cancer surgery and recovery .
LDA showed that, prior to surgery, patients' priorities were primarily in cancer surgery and recovery.
[(0,), (1,), None]
['priorities']-['were']-['cancer', 'surgery', 'and', 'recovery']
[(0,), (17,), None]
1.07 2 489 4
we aim to apply LDA to interview datum collect as part of a prospective , longitudinal study of qol in patient undergo radical cystectomy and urinary diversion for bladder cancer .
We aim to apply LDA to interview data collected as part of a prospective, longitudinal study of QOL in patients undergoing radical cystectomy and urinary diversion for bladder cancer.
[(0,), (1,), None]
['We']-['apply']-['LDA']
['We']-['interview']-['data']
1.0 1 489 3
latent Dirichlet allocation -lrb- lda -rrb- may offer statistical rigor in summarize patient ' concern and cope strategy in a life-threatening illness .
Latent Dirichlet Allocation (LDA) may offer statistical rigor i

Moreover, MCLDA is able to identify the pairing between diagnoses and medications in a record based on the assigned latent groups.
['MCLDA']-['is']-['able']
['MCLDA']-['identify']-['pairing']
['MCLDA']-['identify']-['record']
0.36 2 1112 3
use a real-world research testb that include one million healthcare insurance claim record , we investigate the utility of MCLDA .
Using a real-world research testbed that includes one million healthcare insurance claim records, we investigate the utility of MCLDA.
['we']-['investigate']-['utility', 'of', 'MCLDA']
[(14,), (4,), None]
['that']-['includes']-['healthcare', 'insurance', 'claim', 'records']
0.31 1 1112 7
we evaluation result also show that , in most case , MCLDA outperform alternative method such as logistic regression and the k-nearest-neighbor -lrb- knn -rrb- model for two prediction task , i.e. , medication and diagnosis prediction .
Our evaluation results also show that, in most cases, MCLDA outperforms alternative methods such as log

Both PRSM and WS-PRSM were compared with two established supervised risk stratification algorithms, i.e., logistic regression and support vector machine, and showed the effectiveness of our models in risk stratification of CHD in terms of the Area Under the receiver operating characteristic Curve (AUC) analysis.
['PRSM', 'and', 'WS-PRSM']-['compared', 'with']-['established', 'supervised', 'risk', 'stratification', 'algorithms']
0.27 1 1402 3
method : along this line , this paper propose a novel probabilistic topic modeling framework call probabilistic risk stratification model -lrb- prsm -rrb- base on latent Dirichlet allocation -lrb- lda -rrb- .
Methods: Along this line, this paper proposes a novel probabilistic topic modeling framework called probabilistic risk stratification model (PRSM) based on Latent Dirichlet Allocation (LDA).
1.67 3 1129 0
in healthcare organizational setting , the design of a clinical pathway -lrb- cp -rrb- be challenge since patient follow a particular pathwa

In [9]:
topic_triples = []
print(top_terms[t_idx][:20])
for top_doc in top_docs[t_idx]:
    _idx = top_doc[0]
    if type(parse[_idx]) is str:
        print(_idx, "parse", "err")
        continue
    sents = relation.convert_parse2lemma_sents(parse[_idx])
    sort_idxs, importance, counts = relation.extract_important_sents(sents, [x[0] for x in top_terms[t_idx][:20]], [x[1] for x in top_terms[t_idx][:20]])
    for i in sort_idxs:
        if importance[i]>0:
            sent_tokens = parse[_idx]["sentences"][i]["tokens"]
            sent_deps = parse[_idx]["sentences"][i]["enhancedPlusPlusDependencies"]
            triples = relation.extract_triples_from_sent(sent_deps, sent_tokens, True)
            for triple in triples:
                if None in triple:
                    continue
                topic_triples.append([
                    [sent_tokens[i]["lemma"] for i in triple[0]],
                    [sent_tokens[i]["lemma"] for i in triple[1]],
                    [sent_tokens[i]["lemma"] for i in triple[2]]
                                     ])
scores = []
for t in topic_triples:
    s = relation.evaluate_topic_triple(reduce(lambda a,b:a+b, t), [x[0] for x in top_terms[t_idx][:20]], [x[1] for x in top_terms[t_idx][:20]])
    scores.append(s)
left = 0
for idx in np.argsort(scores)[-1::-1]:
    if scores[idx]>0:
        left+=1
        print(round(scores[idx], 2), 
              "{}-[{}]-{}".format(" ".join(topic_triples[idx][0])," ".join(topic_triples[idx][1])," ".join(topic_triples[idx][2])))
print("total {} triples, left {}".format(len(topic_triples), left))

[('patient', 223.06), ('medical', 143.69), ('health', 143.5), ('clinical', 139.72), ('treatment', 99.44), ('diagnosis', 86.78), ('disease', 81.07), ('risk', 79.0), ('record', 78.67), ('care', 68.06), ('friend', 47.64), ('healthcare', 42.31), ('cancer', 39.36), ('code', 36.28), ('data', 32.74), ('conclusion', 30.77), ('physician', 29.99), ('emr', 25.48), ('ehr', 25.13), ('doctor', 24.57)]
1.88 hospital emr dataset-[consist of]-medical record of patient
1.87 patient record-[contain]-valuable medical information
1.58 propose PRSM-[recognize]-patient clinical state
1.55 exist risk stratification technique-[predict]-overall risk of patient
1.28 patient with disease-[be]-interested in competence
1.28 patient with disease-[be]-interested in ethic
1.28 result-[classify]-heart disease and patient
1.0 result-[classify]-healthy patient
1.0 additional implicit patient need-[use]-qualitative analysis
1.0 patient-[be]-due for visit
1.0 manifestation sub-category-[exist in]-t2dm patient
1.0 470-[read

In [10]:
ps = relation.get_phrases_by_pattern(parse)
sorted(dict(Counter([" ".join(x) for x in ps])).items(), key=lambda x:x[1], reverse=True)

func get_phrases_by_pattern exec time: 0.36412763595581055


[('latent Dirichlet allocation', 1216),
 ('topic model', 692),
 ('experimental result', 431),
 ('topic modeling', 337),
 ('latent topic', 203),
 ('LDA model', 166),
 ('Dirichlet allocation', 158),
 ('short text', 123),
 ('Elsevier B.V. All rights', 108),
 ('sentiment analysis', 103),
 ('dirichlet allocation', 102),
 ('social media', 101),
 ('topic distribution', 93),
 ('social network', 91),
 ('recent year', 86),
 ('large number', 82),
 ('Elsevier Ltd.', 81),
 ('text mining', 79),
 ('Gibbs sampling', 77),
 ('case study', 73),
 ('information retrieval', 70),
 ('probabilistic topic model', 70),
 ('natural language processing', 66),
 ('previous work', 66),
 ('latent Dirichlet Allocation', 65),
 ('large amount', 61),
 ('visual word', 60),
 ('latent Dirichlet allocation model', 58),
 ('novel approach', 57),
 ('extensive experiment', 54),
 ('machine learning', 53),
 ('big datum', 53),
 ('better performance', 53),
 ('new method', 52),
 ('online review', 51),
 ('important role', 51),
 ('langua